In [17]:
import datetime
import pandas as pd

In [18]:
#https://cdn.buenosaires.gob.ar/datosabiertos/datasets/sbase/subte-viajes-molinetes/molinetes-2020.zip
input_file_name = 'subte.csv'

In [19]:
output_file_name = "subte.xlsx"
output_sheet_name = "subte"

In [20]:
df_subte = pd.read_csv(input_file_name)

In [21]:
df_subte.head()

,FECHA,DESDE,HASTA,LINEA,MOLINETE,ESTACION,pax_pagos,pax_pases_pagos,pax_franq,pax_TOTAL
0,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_Acoyte_N_Turn01,Acoyte,1.0,0.0,0.0,1.0
1,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_Carabobo_E_Turn02,Carabobo,6.0,0.0,0.0,6.0
2,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_CBarros_N_Turn03,Castro Barros,3.0,0.0,1.0,4.0
3,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_CBarros_S_Turn02,Castro Barros,2.0,0.0,0.0,2.0
4,01/01/2020,08:00:00,08:15:00,LineaA,LineaA_Congreso_N_Turn03,Congreso,2.0,0.0,0.0,2.0


In [22]:
df_subte.shape

(5781006, 10)

In [23]:
df_subte = df_subte[df_subte['FECHA'].notna()]

In [24]:
df_subte["AÑO"] = pd.DatetimeIndex(df_subte['FECHA']).year

In [25]:
df_subte["MES"] = pd.DatetimeIndex(df_subte['FECHA']).month

In [26]:
df_subte['TIPO'] = 'Subte'

In [27]:
df = df_subte.groupby(['AÑO','MES','TIPO'],as_index=False).sum()[['AÑO','MES','TIPO','pax_TOTAL']].rename({'pax_TOTAL':'TOTAL'},axis=1)

In [28]:
df

,AÑO,MES,TIPO,TOTAL
0,2020,1,Subte,14431260.0
1,2020,2,Subte,14379523.0
2,2020,3,Subte,16665570.0
3,2020,4,Subte,2532868.0
4,2020,5,Subte,2769479.0
5,2020,6,Subte,3566230.0
6,2020,7,Subte,3417198.0
7,2020,8,Subte,3122951.0
8,2020,9,Subte,3629713.0
9,2020,10,Subte,4966696.0


In [29]:
df['TOTAL'] = df['TOTAL'].astype(int)
df['INTERVALO']=pd.cut(df['TOTAL'],10,labels=[1,2,3,4,5,6,7,8,9,10])
df

,AÑO,MES,TIPO,TOTAL,INTERVALO
0,2020,1,Subte,14431260,9
1,2020,2,Subte,14379523,9
2,2020,3,Subte,16665570,10
3,2020,4,Subte,2532868,1
4,2020,5,Subte,2769479,1
5,2020,6,Subte,3566230,1
6,2020,7,Subte,3417198,1
7,2020,8,Subte,3122951,1
8,2020,9,Subte,3629713,1
9,2020,10,Subte,4966696,2


In [30]:
enero = df.loc[ df.MES == 1, ['INTERVALO']].iloc[0,0]
diciembre = df.loc[ df.MES == 12, ['INTERVALO']].iloc[0,0]
diciembre_enero = int(((diciembre - enero)/enero)*100)
if diciembre_enero > 0:
    diciembre_enero = '+' + str(diciembre_enero) + ' %'
else:
    diciembre_enero = str(diciembre_enero) + ' %'

In [31]:
df['DICIEMBRE_ENERO'] = diciembre_enero
df

,AÑO,MES,TIPO,TOTAL,INTERVALO,DICIEMBRE_ENERO
0,2020,1,Subte,14431260,9,-77 %
1,2020,2,Subte,14379523,9,-77 %
2,2020,3,Subte,16665570,10,-77 %
3,2020,4,Subte,2532868,1,-77 %
4,2020,5,Subte,2769479,1,-77 %
5,2020,6,Subte,3566230,1,-77 %
6,2020,7,Subte,3417198,1,-77 %
7,2020,8,Subte,3122951,1,-77 %
8,2020,9,Subte,3629713,1,-77 %
9,2020,10,Subte,4966696,2,-77 %


In [32]:
writer = pd.ExcelWriter(output_file_name, engine='xlsxwriter')

df.to_excel(writer, index=False, sheet_name=output_sheet_name)

workbook  = writer.book
worksheet = writer.sheets[output_sheet_name]

format = workbook.add_format()
format.set_align('center')
format.set_align('vcenter')

worksheet.set_column('A:F', 30, format)

writer.save()